# Temat: Klasyfikacja rodzajów gestów dłoni na podstawie nagranej aktywności mięśni

### Autorzy
Damian Kacperski
Maciej Dmochowski

Grupa 3, Czwartek 8:15 - 10:00

## Opis problemu

Celem projektu jest sklasyfikowanie czterech rodzaji gestów dłoni: kamienia (ściśniętej pięści), papieru (prostej ręki), nożyc (wyprostowany tylko palec środkowy i wskazujący) oraz znaku OK (wyprostowany kciuk). Analizę należy przeprowadzić na podstawie 8 odczytów z 8 czujników EMG aktywności ludzkich mięśni.

## Wczytanie danych

In [ ]:
import pandas as pd

path = 'r./data/'
dt = [pd.read_csv(path + str(file) + '.csv', header=None) for file in range(0, 4)]

columns = ['r' + str(reading) + '_s' + str(sensor) for reading in range(0, 8) for sensor in range(0, 8)] + ['gesture']
for dta in dt:
    dta.columns = columns

data = pd.concat(dt, sort=False)

data.head()

## Podstawowa analiza danych

### Określenie liczby obiektów

In [ ]:
objects_number = len(data.index)

print('Liczba obiektów:', objects_number)

### Określenie liczby klas

In [ ]:
classes_number = len(data['gesture'].unique())

print('Liczba klas:', classes_number)

### Określenie zakresów zmienności poszczególnych atrybutów

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10, 350), dpi = 80)
for r in range(0, 8):
    for s in range(0, 8):
        plt.subplot(64, 1, r*8 + (s+1))
        name = 'r' + str(r) + '_s' + str(s)
        sns.boxplot(x = "gesture", y = name, data = data)

### Określenie wartości statystycznych

In [ ]:
sensors_data = data.drop(['gesture'], axis=1)
sensors_data.describe()

### Określenie poziomu wypełnienia kolumn

In [ ]:
objects_with_missing_columns = data.isna().sum().sum()

print('Liczba obiektów z brakującymi kolumnami:', objects_with_missing_columns)

### Określenie ilości unikalnych danych

In [ ]:
unique_data_count = len(data.drop_duplicates())

print('Ilość unikalnych danych:', unique_data_count)

Dostarczone dane składają się z 11678 obiektów, każdy obiekt jest unikalny i posiada wypełnione wszystkie kolumny tj. pierwsze 64 kolumny to informacje o 8 kolejnych odczytach z 8 czujników, a 65 kolumna to rodzaj gestu, jedna z 4 klas. Zakres zmienności jest szczególnie duży przy odczytach dla klasy "0", czyli gestu pięści, kiedy wszystkie mięśnie są aktywne, natomiast szczególnie mały dla klasy "1", czyli gestu papieru, kiedy mięśnie są rozluźnione.

## Analiza korelacji między zmiennymi

Przeanalizować korelację między zmiennymi

In [ ]:
plt.figure(figsize = (64, 64), dpi = 100)
sns.heatmap(sensors_data.corr(), annot = sensors_data.corr())
plt.show()

Z analizy korelacji możemy odczytać, że istnieje relacja pomiędzy tymi samymi czujnikami w ramach kolejnych odczytów oraz pomiędzy różnymi czujnikami w ramach jednego odczytu. Natomast związek pomiędzy różnymi czujnikami w ramach różnych odczytów jest bardzo słaby.

## Przygotowanie danych do analizy

Dane są w pełni wypełnione dlatego nie ma potrzeby usuwania brakujących danych lub kolumn.

## Analiza podobieństwa przy pomocy algorytmów grupowania

Przeanalizować podobieństwo między danymi przy pomocy poznanych algorytmów grupowania wraz z analizą ilości grup

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, ward, fcluster
from sklearn.cluster import AgglomerativeClustering

ile_grup = 4
sd_copy = sensors_data
attribute_1 = 'r5_s4'
attribute_2 = 'r1_s6'

klasa = data['gesture'].astype('category').cat.codes

for index, linkage in enumerate(('average', 'complete', 'single', 'ward')):
    model = AgglomerativeClustering(linkage=linkage, n_clusters=ile_grup)
    model.fit(sd_copy)
    grupa = model.labels_

    plt.figure(figsize=(15,5))
    
    plt.subplot(1,3,1)
    plt.scatter( x=data[attribute_1], y=data[attribute_2], c=klasa)
    plt.title('dane oryginalne')

    plt.subplot(1,3,2)
    sd_copy['grupa']= model.labels_
    plt.scatter( x=sensors_data[attribute_1], y=sd_copy[attribute_2], c=grupa)
    plt.title('wynik grupowania')

    plt.subplot(1,3,3)
    pomylki = pd.crosstab(data['gesture'], sd_copy['grupa'])
    print(pomylki)
    sns.heatmap(pomylki, annot = pomylki)
    plt.title('macierz pomyłek')
    
    plt.suptitle('Metoda wyznaczania odległości międzygrupowej: ' + linkage, size=17)

Analiza przy pomocy algorytmów grupowania nie przyniosła pozytywnych efektów. Spowodowane jest to tym, że dane każdej z klas mają podobne wartości i wszystkie obiekty są w bliskich odległościach. Grupowanie k-średnich również nie przyniosłoby pozytywnego efektu, ponieważ centroidy grup znajdują się praktycznie w jednym punkcie.

## Testowanie wybranych klasyfikatorów

Testujemy wybrane klasyfikatory pod kątem doboru ich parametrów

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.multiclass import unique_labels

Podział zbioru danych na argumenty i wyniki, podział na dane treningowe i testowe.

In [ ]:
x = data.drop(['gesture'], axis=1)
x = StandardScaler().fit_transform(x)
y = data['gesture']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .30, random_state=38)

Wyświetlanie błędów

In [ ]:
import numpy as np

def view_confusion_matrix(y_true, y_pred, classes,title=None, cmap=plt.cm.Greens):
    if not title:
        title="Macierz błędów"
    conf_mtx = confusion_matrix(y_true, y_pred)

    fig, ax = plt.subplots()
    im = ax.imshow(conf_mtx, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(conf_mtx.shape[1]),
           yticks=np.arange(conf_mtx.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Prawdziwa klasa',
           xlabel='Przewidywana klasa')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = 'd'
    
    thresh = conf_mtx.max() / 2.
    
    for k in range(conf_mtx.shape[0]):
        for l in range(conf_mtx.shape[1]):
            ax.text(k, l, format(conf_mtx[k, l], fmt),
                    ha="center", va="center",
                    color="white" if conf_mtx[k, l] > thresh else "black")
            
    fig.tight_layout()
    
    return ax

### Klasyfikator KNN na podstawie 15 najbliższych

In [ ]:
knn = KNeighborsClassifier(15)

knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
result = knn.score(x_test, y_test)

target_names=['kamień','papier','nożyce','OK']

print("Sprawność ", result, "\n" )
print(classification_report(y_test, y_pred, target_names=target_names))
view_confusion_matrix(y_test, y_pred, classes=target_names)

plt.show()

### Klasyfikator KNN - klasyfikacja na podstawie 100 najbliższych sąsiadów

In [ ]:
knn = KNeighborsClassifier(100)

knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
result = knn.score(x_test, y_test)

target_names=['kamień','papier','nożyce','OK']

print("Sprawność ", result, "\n" )
print(classification_report(y_test, y_pred, target_names=target_names))
view_confusion_matrix(y_test, y_pred, classes=target_names)

plt.show()

Wraz ze wzrostem liczby sasiadów do których jest porównywany punkt sprawność znacząco maleje. Przy 15 sąsiadach przy pierwszej próbie dokładność wynosi jedynie 63% a, przy wzroscie ich do 100 maleje jeszcze o 10% do 53%. Tak mała sprawność jest nie do zakcepotowania. Tak niska skuteczność spowodowana jest wielowymiarowością danych. Wraz ze wzrostem sąsiadów wzrasta również czas uczenia i predykcji.

### Naiwny klasyfikator bayesowski 

In [ ]:
ncb = GaussianNB()

ncb.fit(x_train, y_train)
y_pred = ncb.predict(x_test)
result = ncb.score(x_test, y_test)

target_names=['kamień','papier','nożyce','OK']

print("Sprawność ", result, "\n" )
print(classification_report(y_test, y_pred, target_names=target_names))
view_confusion_matrix(y_test, y_pred, classes=target_names)

plt.show()

Sprawność jaką osiąga ten klasyfikator wynosi około 88% co jest już przyzwoitym wynikiem. Praca na niezależnych predykatorach dobrze widać sprawdza się na danym zbiorze danych - sprawnie się uczy i ma dużą moc predykcyjną.

### Klasyfikator Drzewa Decyzyjnego bez ustawionej głębokości maksymalnej - kryterium 'gini'.

In [ ]:
dtc = DecisionTreeClassifier()

dtc.fit(x_train, y_train)
y_pred = dtc.predict(x_test)
result = dtc.score(x_test, y_test)

target_names=['kamień','papier','nożyce','OK']

print("Sprawność: ", result, "\n" )
print(classification_report(y_test, y_pred, target_names=target_names))
view_confusion_matrix(y_test, y_pred, classes=target_names)

plt.show()

### Klasyfikator Drzewa Decyzyjnego z ustawioną głębokością maksymalną - kryterium 'gini'.

In [ ]:
dtc = DecisionTreeClassifier(max_depth=25)

dtc.fit(x_train, y_train)
y_pred = dtc.predict(x_test)
result = dtc.score(x_test, y_test)

target_names=['kamień','papier','nożyce','OK']

print("Sprawność: ", result, "\n" )
print(classification_report(y_test, y_pred, target_names=target_names))
view_confusion_matrix(y_test, y_pred, classes=target_names)

plt.show()

### Klasyfikator Drzewa Decyzyjnego z ustawioną głębokością maksymalną, kryterium 'entropia'.

In [ ]:
dtc = DecisionTreeClassifier(max_depth=25, criterion='entropy')

dtc.fit(x_train, y_train)
y_pred = dtc.predict(x_test)
result = dtc.score(x_test, y_test)

target_names=['kamień','papier','nożyce','OK']

print("Sprawność: ", result, "\n" )
print(classification_report(y_test, y_pred, target_names=target_names))
view_confusion_matrix(y_test, y_pred, classes=target_names)

plt.show()

Wyniki są do siebie bardzo zbliżone. Dokładność przewidywania wynosi ok 78%. Przy zastosowaniu klasyfikatora drzewa decyzyjnego z ustawioną głębokością maksymalną, oraz kryterium 'gini' skuteczność algorytmu jest o ok 1% lepsza niż bez wyznaczonej głębokości maksymalnej. Mimo wszystko jest to przyzwoity wynik, mimo że zbiór danych jest wielowymiarowy. 
Ustawienie maksymalnej głębokości pozwala uniknąć przeładowania.

## Wnioski
Analizowane dane są pełne i odpowienio uporządkowane. Związek pomiędzy różnymi czujnikami w ramach różnych odczytów jest bardzo słaby. Analiza przy pomocy algorytmów grupowania nie przyniosła pozytywnych efektów, ponieważ wartości danych w klasach są zbyt podobne i obiekty są w zbyt bliskich odległościach. Grupowanie k-średnich też nie dało by akceptowalnego rozwiązania, ponieważ centroidy grup znajdują się /- można by rzec - w jednym punkcie.
Najlepiej działającym klasyfikatorem na danym zbiorze danych okazał się naiwny klasyfikator bayerowski. Sprawność 88% jaką osiągnął jest zadowalający. Praca na niezależnych predykatorach dobrze sprawdza się dla tak różnych danych dla róznych klas. Najczęściej mylone są gesty podobne do siebie np. "nożyczki"- "papier", "ok" - "kamień".
Oceniając wyniki jest możliwa poprawna i całkiem dokładna klasyfikacja gestów z zebranych danych.